## Imports

In [ ]:
import datetime as dt
import pandas as pd
import plotly.graph_objects as go
from hbt import utils_hbt as uh
from pathlib import Path
from plotly.subplots import make_subplots

## Globals

In [ ]:
dt_format = '%d.%m.%Y %H:%M:%S'
save_dir = Path(r"Q:\Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Plots")

## Measurement units

In [ ]:
measurements_units = {
    '432': { # data until 07.06!
        'name': 'Niveau Pumpensumpf PW Hirsen',
        'units': 'cm',
        'color': '',
        'y_axis': 'y1',
    },

    '457': { # data until 06.04!
        'name': 'Niveau Pumpensumpf PW Sonnenbergstrasse',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '458': { # OK
        'name': 'Niveau Einlaufkammer Absetzbecken',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '459': { # OK
        'name': 'RFB 1 - Niveau 1',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '460': { # OK
        'name': 'RFB 1 - Niveau 2',
        'units': 'bar',
        'color': '',
        'y_axis': 'y1',
    },

    '461': { # OK
        'name': 'RFB 2 - Niveau 1',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '462': { # no data!
        'name': 'RFB 2 - Niveau 2',
        'units': 'm',
        'color': '',
        'y_axis': 'y1',
    },

    '463': { # no data!
        'name': 'Trübung Auslaufbauwerk RB Mühlestrasse',
        'units': 'FNU',
        'color': '',
        'y_axis': 'y3',
    },

    '464': { # OK
        'name': 'Trübung Ablauf Absetzbecken',
        'units': 'FNU',
        'color': '',
        'y_axis': 'y3',
    },

    '465': { # OK
        'name': 'Niveau Auslaufbauwerk RB Mühlestrasse',
        'units': 'cm',
        'color': '',
        'y_axis': 'y1',
    },

    '466': { # OK
        'name': 'Durchflussmessung Auslauf SABA',
        'units': 'L/s',
        'color': 'blue',
        'y_axis': 'y2',
    }
}

measurements_units = uh.(measurements_units)

## Rain

In [ ]:
path_rain = Path(r"Q:\Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Regendaten\Buchrain_10min.csv")

rain = uh.read_raindata_agrometeo(path_rain, resolution='10min')

## Pumpwerke

### Daten

In [ ]:
ids = ['432', '457']
# 432: PW Hirsen
# 457: PW Sonnenbergstrasse

dt_from = "01.04.2022 00:00:00"
dt_to = "20.09.2022 00:00:00"
data = {}
for id in ids:
    data.update({id: uh.hbtdb_get_data(hbt_id=id, dt_from=dt_from,
        dt_to=dt_to)})
data['432']['wert'] = data['432']['wert'] / 100 # cm -> m

### Analysis

In [ ]:
lvl_hir = 3.1
pw_hir = data['432']
ent_hir = pw_hir[pw_hir > lvl_hir]
ent_hir = ent_hir[(ent_hir.shift(1) < ent_hir) & (ent_hir.shift(-1) < ent_hir)]
ent_hir.dropna(inplace=True)
print(f'In der Periode wurden {len(ent_hir)} Überläufe detektiert')
fig, ax = plt.subplots()
ax.plot(pw_hir)
ax.scatter(ent_hir.index, ent_hir['wert'], c='r', marker='*')
for i, (idx, row) in enumerate(ent_hir.iterrows()):
    t = (-1)**(i+1)
    t = dt.timedelta(days=t)
    ax.annotate(str(i+1), (idx+t, row['wert']+0.1))
ax.set_xlabel('Datum')
ax.set_ylabel('Niveau [m]')
ax.set_ylim(0,4)
ax.axhline(y=lvl_hir, color='grey', linestyle=':')
ax.annotate(f'Überlauf: {lvl_hir} m', (pw_hir.index[0], lvl_hir*1.02))
fig.set_size_inches(12,5)

In [ ]:
ids = ['432', '457']
# 432: PW Hirsen
# 457: PW Sonnenbergstrasse

dt_from = "01.04.2022 00:00:00"
dt_to = "20.09.2022 00:00:00"
data = {}
for id in ids:
    data.update({id: uh.hbtdb_get_data(hbt_id=id, dt_from=dt_from,
        dt_to=dt_to)})
data['432']['wert'] = data['432']['wert'] / 100 # cm -> m

### Plot

In [ ]:
fig = go.Figure()
id = '432'
fig.add_trace(go.Scatter(x=data[id].index, y=data[id]['wert'],
    name=measurements_units.loc[id]['name'], opacity=1))

id = '457'
fig.add_trace(go.Scatter(x=data[id].index, y=data[id]['wert'],
    name=measurements_units.loc[id]['name']))

fig.update_layout(
    title='Pumpwerke',
    xaxis_title="Datum",
    yaxis_title="Niveau [m]",
)

fig.add_hline(y=3.3, line_color='red', line_dash='dash')

file_path = save_dir / ('Pumpwerke.html')
fig.write_html(file_path)

## SABA

### Parameters

In [ ]:
Dh_rfb1 = 1.74587 # m, average when no rain 17 - 20.06.22
Dh_rfb2 = 0.92158 # m, average when no rain 17 - 20.06.22
A_rfb1 = 150 # m2
A_rfb2 = 150 # m2

rain_ev1_start = [
    '15.06.2022 18:19:36',
    '22.06.2022 21:34:39',
    '24.06.2022 17:39:53',
    '25.06.2022 04:54:57',
]
rain_ev1_start = uh.list_to_dt(rain_ev1_start)

rain_ev2_start = [
    '27.06.2022 20:19:38',
    '01.07.2022 04:07:38',
    '01.07.2022 09:26:38'
]
rain_ev2_start = uh.list_to_dt(rain_ev2_start)

rain_ev1_end = [
    '15.06.2022 18:42:37',
    '22.06.2022 21:55:37',
    '24.06.2022 18:08:53',
    '25.06.2022 05:46:57'
]
rain_ev1_end = uh.list_to_dt(rain_ev1_end)

rain_ev2_end = [
    '27.06.2022 21:21:38',
    '01.7.2022 07:13:38',
    '01.07.2022 15:46:38'
]
rain_ev2_end = uh.list_to_dt(rain_ev2_end)

n1 = len(rain_ev1_start)

### Data

In [ ]:
ids = ['459', '461']

dt_from = dt.strptime("01.10.2021 00:00:00", dt_format)
dt_to = dt.strptime("11.07.2022 00:00:00", dt_format)
data = {}
for id in ids:
        data.update({id: uh.hbtdb_get_data(hbt_id=id, dt_from=dt_from,
            dt_to=dt_to)})
data['459'] = (data['459'] - Dh_rfb1) * (-1)
data['461'] = (data['461'] - Dh_rfb2) * (-1)

### Plot

#### RFB 1

In [ ]:
# Read data
lvl1 = 459
lvl2 = 460
lvl_asb = 458

dt_start = '12.10.2022 10:00:00'
dt_end = '01.11.2022 00:00:00'

ts_lvl1 = uh.hbtdb_get_data(lvl1, dt_start, dt_end)
ts_lvl2 = uh.hbtdb_get_data(lvl2, dt_start, dt_end)
ts_asb = uh.hbtdb_get_data(lvl_asb, dt_start, dt_end)

# Shift and invert data
ts_lvl1 = (ts_lvl1 - 1.73) * -1 * 100
ts_asb = (ts_asb - 4.43) * -1

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts_lvl1.index, y=ts_lvl1['wert'],
    name='Niveau RFB1'))
fig.add_trace(go.Scatter(x=ts_asb.index, y=ts_asb['wert'],
    name='Niveau ASB'))
fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Niveau [cm]')
fig.update_layout(title='Niveau RFB1')

save_path = save_dir / 'rfb1.html'
fig.write_html(save_path)

In [ ]:
dates = ['2022-06-14', '2022-06-27', '2022-07-04']
colors = ['red', 'green']

fig = make_subplots(specs=[[{"secondary_y": False}]])
for i, id in enumerate(ids):
    fig.add_trace(go.Scatter(x=data[id].index, y=data[id]['wert'],
        name=measurements_units.loc[id]['name'], marker_color=colors[i]))

fig.add_vrect(x0=dates[0], x1=dates[1], row="all", col=1, 
    annotation_text="Beschickung RFB 1", annotation_position="top left",
    fillcolor=colors[0], opacity=0.2, line_width=1, line_color='black')

fig.add_vrect(x0=dates[1], x1=dates[2], row="all", col=1, 
    annotation_text="Beschickung RFB 2", annotation_position="top left",
    fillcolor=colors[1], opacity=0.2, line_width=1, line_color='black')

ax = -30
ay = -30
for i, rev in enumerate(rain_ev1_start):
    fig.add_annotation(x=rev, y=data['459'].loc[rev].values[0],
        text=f'Ereignis {i+1}', showarrow=True, ax=ax, ay=ay)

for i, rev in enumerate(rain_ev2_start):
    if i == len(rain_ev2_end) - 1:
        ax = 0
        ay = -60
    fig.add_annotation(x=rev, y=data['461'].loc[rev].values[0],
        text=f'Ereignis {n1+1+i}', showarrow=True, ax=ax, ay=ay)

fig.update_layout(
    title='Niveaus SABA',
    xaxis_title="Datum",
    yaxis_title="Niveau [m]",
)

file_path = save_dir / ('Niveaus-SABA.html')
fig.write_html(file_path)

### Calculations

In [ ]:
def filter_leistung(dt_start, dt_end, ts_data):
    h_start = ts_data.loc[dt_start].values[0]
    h_end = ts_data.loc[dt_end].values[0]
    delta_t = (dt_end - dt_start).total_seconds() / 60 # min
    delta_h = (h_start - h_end) * 1e3

    return delta_h / delta_t

columns = ['Ereignis', 'Start', 'Ende', 'Filter Leistung']
df_res1 = pd.DataFrame(columns=columns)
for i, el in enumerate(rain_ev1_start):
    dt_start = rain_ev1_start[i]
    dt_end = rain_ev1_end[i]
    fl = filter_leistung(dt_start, dt_end, data['459'])
    df_res1.loc[i] = [f'Ereignis {i+1}', dt_start.strftime(dt_format),
        dt_end.strftime(dt_format), f'{fl:0.1f}']
df_res1.set_index('Ereignis', inplace=True)

df_res2 = pd.DataFrame(columns=columns)
for i, el in enumerate(rain_ev2_start):
    dt_start = rain_ev2_start[i]
    dt_end = rain_ev2_end[i]
    fl = filter_leistung(dt_start, dt_end, data['461'])
    df_res2.loc[i] = [f'Ereignis {n1+1+i}', dt_start.strftime(dt_format),
        dt_end.strftime(dt_format), f'{fl:0.1f}']
df_res2.set_index('Ereignis', inplace=True)

In [ ]:
df_res1